In [ ]:
# 02. Enriquecimento de Dados: chuva

1. Bibliotecas e DataFrame

In [1]:
import requests
import pandas as pd
import os

df = pd.read_parquet("../dados/intermediarios/acidentes_tratado.parquet")

2. Coordenadas por região

In [3]:
coord = {
    "NORTE":  (-29.987, -51.165),
    "LESTE":  (-30.040, -51.160),
    "CENTRO": (-30.027, -51.220),
    "SUL":    (-30.120, -51.230)
}

anos = [2020, 2021, 2022, 2023, 2024]
dfs_all = []

os.makedirs("../dados/intermediarios/clima", exist_ok=True)

3. Cria função para API (por região, por ano para não quebrar)

In [4]:
# Consulta API Open Meteo
def chuva_regiao_ano(regiao, lat, lon, ano):
    data_inicio = f"{ano}-01-01"
    data_fim = f"{ano}-12-31"
    
    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={data_inicio}&end_date={data_fim}"
        "&hourly=precipitation"
        "&timezone=America/Sao_Paulo"
    )
    
    resposta = requests.get(url).json()
    
    if "hourly" in resposta and "time" in resposta["hourly"]:
        df_meteo = pd.DataFrame({
            "time": pd.to_datetime(resposta["hourly"]["time"], errors="coerce"),
            "precipitation": resposta["hourly"]["precipitation"],
        })
        df_meteo["regiao"] = regiao
        return df_meteo
    else:
        print(f"⚠️ Sem dados para {regiao}-{ano}")
        return pd.DataFrame(columns=["time","precipitation","regiao"])


4. Chama API em Loop

In [5]:
for ano in anos:
    for regiao, (lat, lon) in coord.items():
        print(f"📥 Baixando clima {regiao}-{ano}...")
        df_meteo = chuva_regiao_ano(regiao, lat, lon, ano)

        if not df_meteo.empty:
            fname = f"../dados/intermediarios/clima/clima_{regiao}_{ano}.parquet"
            df_meteo.to_parquet(fname, index=False, engine="pyarrow")
            print(f"✅ salvo {fname} ({len(df_meteo)} linhas)")
            dfs_all.append(df_meteo)

df_meteo_regioes = pd.concat(dfs_all, ignore_index=True)
df_meteo_regioes.to_parquet("../dados/intermediarios/clima/clima_2020_2024.parquet", index=False, engine="pyarrow")
print("📦 Arquivo final consolidado salvo.")

📥 Baixando clima NORTE-2020...


✅ salvo ../dados/intermediarios/clima/clima_NORTE_2020.parquet (8784 linhas)
📥 Baixando clima LESTE-2020...
✅ salvo ../dados/intermediarios/clima/clima_LESTE_2020.parquet (8784 linhas)
📥 Baixando clima CENTRO-2020...
✅ salvo ../dados/intermediarios/clima/clima_CENTRO_2020.parquet (8784 linhas)
📥 Baixando clima SUL-2020...
✅ salvo ../dados/intermediarios/clima/clima_SUL_2020.parquet (8784 linhas)
📥 Baixando clima NORTE-2021...
✅ salvo ../dados/intermediarios/clima/clima_NORTE_2021.parquet (8760 linhas)
📥 Baixando clima LESTE-2021...
✅ salvo ../dados/intermediarios/clima/clima_LESTE_2021.parquet (8760 linhas)
📥 Baixando clima CENTRO-2021...
✅ salvo ../dados/intermediarios/clima/clima_CENTRO_2021.parquet (8760 linhas)
📥 Baixando clima SUL-2021...
✅ salvo ../dados/intermediarios/clima/clima_SUL_2021.parquet (8760 linhas)
📥 Baixando clima NORTE-2022...
✅ salvo ../dados/intermediarios/clima/clima_NORTE_2022.parquet (8760 linhas)
📥 Baixando clima LESTE-2022...
✅ salvo ../dados/intermediarios/

KeyboardInterrupt: 